In [ ]:
Here is the full **Google Colab notebook code** combined into a single, executable script. You can copy and paste this directly into a Colab notebook cell to run the entire **Retrieval-Augmented Generation (RAG)** demo.

> 🔐 **Before running:**
- Replace `"your_huggingface_api_token"` with your actual Hugging Face token from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
- Upload a PDF file when prompted after running the notebook

---

```python
# Step 1: Install required libraries
!pip install -q langchain faiss-cpu transformers sentence-transformers pypdf openai

# Step 2: Import libraries
import os
from google.colab import files
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub

# Step 3: Upload and load PDF document
print("Please upload a PDF file.")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
loader = PyPDFLoader(file_name)
documents = loader.load()

# Split document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Step 4: Create embeddings and build vector store
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
db = FAISS.from_documents(texts, embeddings)

# Step 5: Load a pre-trained language model (Hugging Face Hub)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_huggingface_api_token"
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature": 0.5, "max_length": 512}
)

# Step 6: Build the RAG chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

# Step 7: Define question function and ask sample question
def ask_question(question):
    response = qa_chain.run(question)
    print("Answer:", response)

ask_question("What are the key findings in this document?")
```

---

This complete notebook performs the following:
- Installs necessary dependencies
- Loads and processes a user-uploaded PDF
- Splits content into chunks
- Embeds and stores them using FAISS
- Uses a hosted LLM via Hugging Face
- Sets up and runs a RAG-based QA chain

You can modify the `ask_question()` call to test different queries based on your uploaded document.